In [1]:
# if kernel crashes, make sure pywin32 and pipywin32 are installed. 
# Followed instructions here: https://github.com/jupyter/notebook/issues/4909 
import win32api

In [2]:

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import microdf as mdf
import os
import us



In [5]:
# Import data from Ipums
person_raw = pd.read_csv("cps_00041.csv.gz")

# Create copy and lower column names
person = person_raw.copy(deep=True)
person.columns = person.columns.str.lower()
person.asecwt /= 3

# Crate booleans for demographics
person["adult"] = person.age > 17
person["child"] = person.age < 18

person["black"] = person.race == 200
person["white_non_hispanic"] = (person.race == 100) & (person.hispan == 0)
person["hispanic"] = (person.hispan > 1) & person.hispan < 700
person["pwd"] = person.diffany == 2
person["non_citizen"] = person.citizen == 5
person["non_citizen_child"] = (person.citizen == 5) & (person.age < 18)
person["non_citizen_adult"] = (person.citizen == 5) & (person.age > 17)

# Remove NIUs
person["taxinc"].replace({9999999: 0}, inplace=True)
person["adjginc"].replace({99999999: 0}, inplace=True)
person["incss"].replace({999999: 0}, inplace=True)
person["incssi"].replace({999999: 0}, inplace=True)
person["incunemp"].replace({99999: 0}, inplace=True)
person["incunemp"].replace({999999: 0}, inplace=True)
person["ctccrd"].replace({999999: 0}, inplace=True)
person["actccrd"].replace({99999: 0}, inplace=True)
person["eitcred"].replace({9999: 0}, inplace=True)
person["fica"].replace({99999: 0}, inplace=True)
person["fedtaxac"].replace({99999999: 0}, inplace=True)
person["stataxac"].replace({9999999: 0}, inplace=True)

In [13]:
# Change fip codes to state names
person["statefip"]=person["statefip"].astype(str)
person["statefip"]=person["statefip"].apply(lambda x: us.states.lookup(x))
person.statefip.value_counts()

array([<State:Maine>, <State:New Hampshire>, <State:Vermont>,
       <State:Massachusetts>, <State:Rhode Island>, None,
       <State:New York>, <State:New Jersey>, <State:Pennsylvania>,
       <State:Ohio>, <State:Indiana>, <State:Illinois>, <State:Michigan>,
       <State:Wisconsin>, <State:Minnesota>, <State:Iowa>,
       <State:Missouri>, <State:North Dakota>, <State:South Dakota>,
       <State:Nebraska>, <State:Kansas>, <State:Delaware>,
       <State:Maryland>, <State:District of Columbia>, <State:Virginia>,
       <State:West Virginia>, <State:North Carolina>,
       <State:South Carolina>, <State:Georgia>, <State:Florida>,
       <State:Kentucky>, <State:Tennessee>, <State:Mississippi>,
       <State:Louisiana>, <State:Oklahoma>, <State:Texas>,
       <State:Montana>, <State:Idaho>, <State:Wyoming>,
       <State:New Mexico>, <State:Utah>, <State:Nevada>,
       <State:Washington>, <State:Oregon>, <State:Hawaii>], dtype=object)

In [3]:
us.states.mapping('fips','name')

{'01': 'Alabama',
 '02': 'Alaska',
 '04': 'Arizona',
 '05': 'Arkansas',
 '06': 'California',
 '08': 'Colorado',
 '09': 'Connecticut',
 '10': 'Delaware',
 '12': 'Florida',
 '13': 'Georgia',
 '15': 'Hawaii',
 '16': 'Idaho',
 '17': 'Illinois',
 '18': 'Indiana',
 '19': 'Iowa',
 '20': 'Kansas',
 '21': 'Kentucky',
 '22': 'Louisiana',
 '23': 'Maine',
 '24': 'Maryland',
 '25': 'Massachusetts',
 '26': 'Michigan',
 '27': 'Minnesota',
 '28': 'Mississippi',
 '29': 'Missouri',
 '30': 'Montana',
 '31': 'Nebraska',
 '32': 'Nevada',
 '33': 'New Hampshire',
 '34': 'New Jersey',
 '35': 'New Mexico',
 '36': 'New York',
 '37': 'North Carolina',
 '38': 'North Dakota',
 '39': 'Ohio',
 '40': 'Oklahoma',
 '41': 'Oregon',
 '42': 'Pennsylvania',
 '44': 'Rhode Island',
 '45': 'South Carolina',
 '46': 'South Dakota',
 '47': 'Tennessee',
 '48': 'Texas',
 '49': 'Utah',
 '50': 'Vermont',
 '51': 'Virginia',
 '53': 'Washington',
 '54': 'West Virginia',
 '55': 'Wisconsin',
 '56': 'Wyoming',
 '60': 'American Samoa',
 '6